Script to query data from Cell Census.
    
    
SOMA = STACKS of matrices, annotated: https://github.com/single-cell-data/SOMA/blob/main/abstract_specification.md

CELLxGENE dataset schema: https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/3.0.0/schema.md

Helpful links:
https://github.com/chanzuckerberg/cell-census/blob/main/api/python/notebooks/api_demo/census_query_extract.ipynb

Future helpful link:
Interfacing pytorch models with anndata: https://anndata.readthedocs.io/en/latest/tutorials/notebooks/annloader.html

In [5]:
import cell_census
import anndata as ad

In [2]:
census = cell_census.open_soma(census_version="latest")


In [3]:
# Define a simple obs-axis query for all cells where tissue is UBERON:0001264 and sex is PATO:0000383.
adata = cell_census.get_anndata(
    census,
    "Homo sapiens",
    obs_value_filter="tissue_ontology_term_id=='UBERON:0002048' and sex_ontology_term_id=='PATO:0000383' and cell_type_ontology_term_id in ['CL:0002063', 'CL:0000499']",
)

display(adata)

AnnData object with n_obs × n_vars = 119269 × 60664
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_general', 'tissue_general_ontology_term_id'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length'

In ```n_obs``` there are a few ontology related terms. One of these might be our target variable, perhaps cell_type?

- ```cell_type_ontology_term_id``` 
- ```development_stage_ontology_term_id``` 
- ```disease_ontology_term_id``` 
- ```self_reported_ethnicity_ontology_term_id``` 
- ```sex_ontology_term_id``` 
- ```tissue_ontology_term_id``` 
- ```tissue_general_ontology_term_id```

In [4]:
type(adata)

anndata._core.anndata.AnnData

In [6]:
shape = ad.AnnData(adata)
print(shape)

AnnData object with n_obs × n_vars = 119269 × 60664
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_general', 'tissue_general_ontology_term_id'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length'


In [7]:
adata.obs_names

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '119259', '119260', '119261', '119262', '119263', '119264', '119265',
       '119266', '119267', '119268'],
      dtype='object', length=119269)

In [8]:
adata.var_names

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '60654', '60655', '60656', '60657', '60658', '60659', '60660', '60661',
       '60662', '60663'],
      dtype='object', length=60664)

## Both adata.obs and adata.vars are Pandas DataFrames

In [10]:
adata.var

,soma_joinid,feature_id,feature_name,feature_length
0,0,ENSG00000238009,RP11-34P13.7,3726
1,1,ENSG00000279457,WASH9P,1397
2,2,ENSG00000228463,AP006222.1,8224
3,3,ENSG00000237094,RP4-669L17.4,6204
4,4,ENSG00000230021,RP11-206L10.17,5495
...,...,...,...,...
60659,60659,ENSG00000288699,RP11-182N22.10,654
60660,60660,ENSG00000288700,RP11-22E12.2,6888
60661,60661,ENSG00000288710,RP11-386G11.12,2968
60662,60662,ENSG00000288711,AP000326.5,1307


In [11]:
adata.obs

,soma_joinid,dataset_id,assay,assay_ontology_term_id,cell_type,cell_type_ontology_term_id,development_stage,development_stage_ontology_term_id,disease,disease_ontology_term_id,...,is_primary_data,self_reported_ethnicity,self_reported_ethnicity_ontology_term_id,sex,sex_ontology_term_id,suspension_type,tissue,tissue_ontology_term_id,tissue_general,tissue_general_ontology_term_id
0,1947485,97a17473-e2b1-4f31-a544-44a60773e2dd,10x 3' v3,EFO:0009922,type II pneumocyte,CL:0002063,59-year-old human stage,HsapDv:0000153,normal,PATO:0000461,...,False,European,HANCESTRO:0005,female,PATO:0000383,cell,lung,UBERON:0002048,lung,UBERON:0002048
1,1947486,97a17473-e2b1-4f31-a544-44a60773e2dd,10x 3' v3,EFO:0009922,type II pneumocyte,CL:0002063,59-year-old human stage,HsapDv:0000153,normal,PATO:0000461,...,False,European,HANCESTRO:0005,female,PATO:0000383,cell,lung,UBERON:0002048,lung,UBERON:0002048
2,1947489,97a17473-e2b1-4f31-a544-44a60773e2dd,10x 3' v3,EFO:0009922,type II pneumocyte,CL:0002063,59-year-old human stage,HsapDv:0000153,normal,PATO:0000461,...,False,European,HANCESTRO:0005,female,PATO:0000383,cell,lung,UBERON:0002048,lung,UBERON:0002048
3,1947491,97a17473-e2b1-4f31-a544-44a60773e2dd,10x 3' v3,EFO:0009922,type II pneumocyte,CL:0002063,59-year-old human stage,HsapDv:0000153,normal,PATO:0000461,...,False,European,HANCESTRO:0005,female,PATO:0000383,cell,lung,UBERON:0002048,lung,UBERON:0002048
4,1947494,97a17473-e2b1-4f31-a544-44a60773e2dd,10x 3' v3,EFO:0009922,type II pneumocyte,CL:0002063,59-year-old human stage,HsapDv:0000153,normal,PATO:0000461,...,False,European,HANCESTRO:0005,female,PATO:0000383,cell,lung,UBERON:0002048,lung,UBERON:0002048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119264,35694911,4023a2bc-6325-47db-bfdf-9639e91042c2,10x 5' v1,EFO:0011025,type II pneumocyte,CL:0002063,15th week post-fertilization human stage,HsapDv:0000052,normal,PATO:0000461,...,False,unknown,unknown,female,PATO:0000383,cell,lung,UBERON:0002048,lung,UBERON:0002048
119265,35694942,4023a2bc-6325-47db-bfdf-9639e91042c2,10x 5' v1,EFO:0011025,type II pneumocyte,CL:0002063,20th week post-fertilization human stage,HsapDv:0000057,normal,PATO:0000461,...,False,unknown,unknown,female,PATO:0000383,cell,lung,UBERON:0002048,lung,UBERON:0002048
119266,35694943,4023a2bc-6325-47db-bfdf-9639e91042c2,10x 5' v1,EFO:0011025,type II pneumocyte,CL:0002063,20th week post-fertilization human stage,HsapDv:0000057,normal,PATO:0000461,...,False,unknown,unknown,female,PATO:0000383,cell,lung,UBERON:0002048,lung,UBERON:0002048
119267,35694955,4023a2bc-6325-47db-bfdf-9639e91042c2,10x 5' v1,EFO:0011025,type II pneumocyte,CL:0002063,20th week post-fertilization human stage,HsapDv:0000057,normal,PATO:0000461,...,False,unknown,unknown,female,PATO:0000383,cell,lung,UBERON:0002048,lung,UBERON:0002048


In [16]:
print(adata.obs_names[:10])

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='object')


In [21]:
# return a dataframe
adata_df = adata.obs['cell_type']#.to_df()

In [22]:
adata_df.head()

0    type II pneumocyte
1    type II pneumocyte
2    type II pneumocyte
3    type II pneumocyte
4    type II pneumocyte
Name: cell_type, dtype: object

In [17]:
adata_df.shape

(119269, 60664)